# **P3：使用 Python 录音**

可以将树莓派外接一个 USB 麦克风，用来录音

五星参考教程：[Recording Audio on the Raspberry Pi with Python and a USB Microphone](https://makersportal.com/blog/2018/8/23/recording-audio-on-the-raspberry-pi-with-python-and-a-usb-microphone) (讲解非常清晰，感谢作者[Joshua Hrisko](https://makersportal.com/blog?author=59b036fc6073554c1cfffef7)）

基于 **pyaudio** 库，主页[在此](https://people.csail.mit.edu/hubert/pyaudio/)


</br>

In [2]:
import pyaudio  #https://people.csail.mit.edu/hubert/pyaudio/
import wave     # python 自带包，用于读取或写入 wav 格式音频文件

# linux 要先在 pulseaudio 中将录音的输入设备设置为内置话筒（可以安装使用 pavucontrol），否则有可能生成文件为空

form_1 = pyaudio.paInt16            # 16-bit 采样分辨率
chans = 1                           # 单通道
samp_rate = 44100                   # 采样率
chunk = 4096                        # 2^12 一个 buffer 中存储多少个 sample
#dev_index = 5                       # 选用的音频输入设备（内置麦克风，或是外接的 USB 麦克风）
record_secs = 3                     # 录音时长（秒）
wav_output_filename = 'test0.wav'  # 录音文件名

audio = pyaudio.PyAudio()           # pyaudio 生成一个实例

# 创建 pyaudio 流
stream = audio.open(format=form_1, rate = samp_rate, channels = chans,\
                   input = True,\
                   frames_per_buffer = chunk)
print("开始录音")

frames = []

# 重复将 3s 内流经 buffer 的数字音频数据拼接存入 frames 列表中
for ii in range(0,int((samp_rate/chunk)*record_secs)):
    data = stream.read(chunk)
    frames.append(data)
    
print("停止录音")

# 停止流，关闭并终止 pyaudio 实例
stream.close()
audio.terminate()

# 将 frames 列表中的数据以 wav 文件格式保存在本地
wavefile = wave.open(wav_output_filename, 'wb')
wavefile.setnchannels(chans)
wavefile.setsampwidth(audio.get_sample_size(form_1))
wavefile.setframerate(samp_rate)
wavefile.writeframes(b''.join(frames))
wavefile.close()

开始录音
停止录音


In [3]:
import pyaudio  #https://people.csail.mit.edu/hubert/pyaudio/
import wave     # python 自带包，用于读取或写入 wav 格式音频文件
import numpy as np

# linux 要先在 pulseaudio 中将录音的输入设备设置为内置话筒（可以安装使用 pavucontrol），否则有可能生成文件为空

form_1 = pyaudio.paInt16            # 16-bit 采样分辨率
chans = 1                           # 单通道
samp_rate = 44100                   # 采样率
chunk = 4096                        # 2^12 一个 buffer 中存储多少个 sample
#dev_index = 5                       # 选用的音频输入设备（内置麦克风，或是外接的 USB 麦克风）
record_secs = 0.3                     # 录音时长（秒）
wav_output_filename = 'test0.wav'  # 录音文件名

audio = pyaudio.PyAudio()           # pyaudio 生成一个实例

# 创建 pyaudio 流
stream = audio.open(format=form_1, rate = samp_rate, channels = chans,\
                   input = True,\
                   frames_per_buffer = chunk)
print("开始录音")

frames = []
peaks = []

# 重复将 3s 内流经 buffer 的数字音频数据拼接存入 frames 列表中
for ii in range(0,int((samp_rate/chunk)*record_secs)):
    data = np.fromstring(stream.read(chunk),dtype=np.int16)
    peak = np.average(np.abs(data))*2
    bars = "#"*int(50*peak/2**16)
    frames.append(data)
    peaks.append(peak)
    print("%04d %05d %s"%(ii,peak, bars))
print("停止录音")

# 停止流，关闭并终止 pyaudio 实例
stream.close()
audio.terminate()



开始录音
0000 09866 #######


<ipython-input-3-b3a83823d1bb>:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(stream.read(chunk),dtype=np.int16)


0001 05695 ####
0002 08244 ######
停止录音


In [19]:

x=frames[2]/2**15

In [23]:
import scipy.fftpack as scp

In [32]:
y = np.abs(scp.fft(x))[0:2048]

In [36]:
freq = np.arange(0, 1+1.0/2048, 1.0/2048)*44100/2

In [39]:
freq.shape[0]

2049

In [21]:
energy = np.mean(x**2)
int(energy/0.02*80)+1

92

</br>

### **附**|

关于 **chunk/buffer** ：为了快速计算与临时存储音频流数据，以一定单位片段式截取数据流用来判断或分析，进而存储或丢弃或应用算法做数据分析等，[参看这里的回答](https://dsp.stackexchange.com/questions/13728/what-are-chunks-when-recording-a-voice-signal)



In [ ]:
# 使用 pyaudio 查看系统内的音频设备
p = pyaudio.PyAudio()
for ii in range(p.get_device_count()):
    print(p.get_device_info_by_index(ii).get('name'))